In [21]:
import pandas as pd
df = pd.read_csv('public.csv')

## Ajuste de dados e preenchimento de Imagens e Metar nulos

In [22]:
import re
# Preenchendo dados reais nulos por dados previstos
    # Alguns casos podem nao possuir tanto dados reais quanto previstos
df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), 'metaf']

# Substituir o texto METAF por METAR
df['metar'] = df['metar'].apply(lambda txt: re.sub('METAF', 'METAR', str(txt)))

# Funcao para ajustar dados meteorologicos com falta de informacao temporal
def metar_fix(txt):
    txt = str(txt)
    if txt != None and txt != 'nan':  
        first_position = txt.find('SB')
        second_position = txt[first_position+5:].find(' ')
        final_position = first_position+5+second_position
        if txt[final_position-1] != 'Z':
            txt = txt[:final_position] + 'Z' + txt[final_position:]
        return txt

df['metar'] = df['metar'].apply(metar_fix)

'http://satelite.cptec.inpe.br/repositoriogoes/goes16/goes16_web/ams_ret_ch11_baixa/2022/06/S11635384_202206010100.jpg'

In [23]:
# Funcao para preencher as urls de imagem nulas baseado na proximidade por data, threshold de 3 horas 
from datetime import timedelta

df['hora_ref'] = pd.to_datetime(df['hora_ref'])

def find_nearest_image(date, df_full):
    window = timedelta(hours=3)
    datas_proximas = (df_full['hora_ref']-date).abs()
    if (datas_proximas <= window).sum() > 0:
        nearest_row = df_full.iloc[datas_proximas.argsort()[0]]
        return nearest_row['url_img_satelite']
    
df_full = df.loc[~df['url_img_satelite'].isna()].drop_duplicates()

df.loc[df['url_img_satelite'].isna(), 'url_img_satelite'] = df.loc[df['url_img_satelite'].isna(), 'hora_ref'].apply(lambda x: find_nearest_image(x, df_full))

In [24]:
# Funcao para preencher o metar baseado na proximidade por data e aeroporto, threshold de 3 horas 

def find_nearest_metar(date, airport, df_full):
    # print(date, airport)
    window = timedelta(hours=3)
    datas_proximas = (df_full.loc[df_full['origem'] == airport, 'hora_ref']-date).abs()
    if (datas_proximas <= window).sum() > 0:
        nearest_row = df_full.iloc[datas_proximas.argsort().tolist()[0]]
        return nearest_row['metar']
    return None

df_full = df.loc[~df['metar'].isna()].drop_duplicates()

df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), ['hora_ref', 'origem']].apply(lambda x: find_nearest_metar(x[0], x[1], df_full), axis=1)

C:\Users\pedro\AppData\Local\Temp\ipykernel_33500\1887291713.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), ['hora_ref', 'origem']].apply(lambda x: find_nearest_metar(x[0], x[1], df_full), axis=1)
C:\Users\pedro\AppData\Local\Temp\ipykernel_33500\1887291713.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), ['hora_ref', 'origem']].apply(lambda x: find_nearest_metar(x[0], x[1], df_full), axis=1)
C:\Users\pedro\AppData\Local\Temp\ipykernel_33500\1887291713.py:12: FutureWarning: Serie

## Criacao do conjuto de treino

In [97]:
# Importando a tabela de metar construida em metar.ipynb
df_metar = pd.read_csv('metar.csv')

# Removendo os casos de metar e imagens nulos restantes (nao fazem parte do conjunto de teste)
df_model = df.loc[~df['metar'].isna() & ~df['url_img_satelite'].isna()]

# Isolando o nome do arquivo de imagem
df_model['img'] = df_model['url_img_satelite'].apply(lambda txt: txt.split('/')[-1][:-4])

C:\Users\pedro\AppData\Local\Temp\ipykernel_33500\1677367905.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['img'] = df_model['url_img_satelite'].apply(lambda txt: txt.split('/')[-1][:-4])


In [99]:
# Criando uma tabela de latitude e longitude para os aeroportos
df_latitudes = df_metar[['station_id', 'latitude', 'longitude']].drop_duplicates()

# Jutando o dataframe orignal com o de metar, removendo dados que nao serao utilizados
df_model = df_model.merge(df_metar, how='left', left_on='metar', right_on='meta')
df_model.drop(['hora_ref', 'url_img_satelite', 'station_id', 'meta', 'metar', 'metaf', 'date', 'latitude', 'longitude'], axis=1, inplace=True)

# Inserindo as latidudes e longitudes para a origem e destino, e removendo os mesmos
df_model = df_model.merge(df_latitudes, how='left', left_on='origem', right_on='station_id').rename(columns={'latitude':'origem_latitude', 'longitude':'origem_longitude'}).drop(['station_id'], axis=1)
df_model = df_model.merge(df_latitudes, how='left', left_on='destino', right_on='station_id').rename(columns={'latitude':'destino_latitude', 'longitude':'destino_longitude'}).drop(['station_id'], axis=1)
df_model.drop(['origem', 'destino'], axis=1, inplace=True)

In [100]:
# Separacao do conjunto de treino e de teste
df_test = df_model.loc[df_model['espera'].isna()]
df_train = df_model.loc[~df_model['espera'].isna()]

## Treinamento do modelo

In [101]:
import numpy as np
from sklearn.model_selection import train_test_split

# Separando o conjunto de treino em validacao e treino
X = df_train.drop(['flightid', 'espera'], axis=1)
y = df_train['espera']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [104]:
X_train

,prev_troca_cabeceira,troca_cabeceira_hora_anterior,img,elevation,wind_direction,wind_speed,visibility,low_cloud_level,cloud_coverage,air_temperature,...,current_wx1_symbol,current_wx2_symbol,current_wx3_symbol,air_pressure_at_sea_level,eastward_wind,northward_wind,origem_latitude,origem_longitude,destino_latitude,destino_longitude
8386,0,0,S11635384_202206120700,828.0,170.0,2.0,9999.0,4000.0,6.0,18.0,...,0.0,0.0,0.0,1015.38,-0.347296,1.969616,-23.42,-46.47,-19.63,-43.97
24189,1,0,S11635384_202207030800,661.0,120.0,6.0,10000.0,2500.0,0.0,16.0,...,0.0,0.0,0.0,1022.55,-5.196152,3.000000,-15.87,-47.92,-23.00,-47.12
125218,0,0,S11635384_202211071100,19.0,250.0,2.0,9999.0,2000.0,6.0,25.0,...,0.0,0.0,0.0,1014.22,1.879385,0.684040,-23.62,-46.63,-8.13,-34.92
106215,0,0,S11635384_202210142100,1061.0,100.0,9.0,9999.0,4900.0,4.0,31.0,...,0.0,0.0,0.0,1010.13,-8.863270,1.562834,-23.42,-46.47,-15.87,-47.92
74098,0,1,S11635384_202209032100,6.0,110.0,10.0,9999.0,2000.0,4.0,26.0,...,0.0,0.0,0.0,1017.27,-9.396926,3.420201,-8.13,-34.92,-12.90,-38.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0,0,S11635384_202210311800,1061.0,330.0,2.0,9999.0,3500.0,6.0,24.0,...,0.0,0.0,0.0,1008.89,1.000000,-1.732051,-23.62,-46.63,-15.87,-47.92
103694,1,0,S11635384_202210111700,661.0,50.0,9.0,10000.0,3600.0,0.0,28.0,...,0.0,0.0,0.0,1015.33,-6.894400,-5.785088,-22.88,-43.17,-23.00,-47.12
131932,1,0,S11635384_202211151200,828.0,50.0,4.0,9999.0,1000.0,6.0,23.0,...,0.0,0.0,0.0,1015.74,-3.064178,-2.571150,-23.42,-46.47,-19.63,-43.97
146867,0,0,S11635384_202212021400,750.0,80.0,4.0,10000.0,3000.0,0.0,27.0,...,0.0,0.0,0.0,1011.95,-3.939231,-0.694593,-8.13,-34.92,-23.42,-46.47
